In [ ]:
!pip install -q together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 15.5 MB/s eta 0:00:00


In [ ]:
from together import Together
import numpy as np
import pandas as pd
import re
import os
import copy as cp
from tqdm import tqdm
import time

from google.colab import drive, userdata
drive.mount('/content/drive', force_remount=True)
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')
together = Together(api_key=TOGETHER_API_KEY)

Mounted at /content/drive


In [ ]:
courses_numbers = ['544', '566', '585', '596', '599', '626', '677', '699']
course_names = ['Applied Natural Language Processing (NLP)', 'Deep Learning and its Applications (DL)', 'Database Systems (database)', 'Scientific Computing and Visualization',
         'Distributed Systems', 'Text as Data', 'Advanced Computer Vision (CV)', 'Robotic Perception (Robotics)']

full_course_info = dict(zip(['CSCI' + num for num in courses_numbers], ['CSCI' + num + ' ' + name for num, name in zip(courses_numbers, course_names)]))

In [ ]:
full_course_info

{'CSCI544': 'CSCI544 Applied Natural Language Processing (NLP)',
 'CSCI566': 'CSCI566 Deep Learning and its Applications (DL)',
 'CSCI585': 'CSCI585 Database Systems (database)',
 'CSCI596': 'CSCI596 Scientific Computing and Visualization',
 'CSCI599': 'CSCI599 Distributed Systems',
 'CSCI626': 'CSCI626 Text as Data',
 'CSCI677': 'CSCI677 Advanced Computer Vision (CV)',
 'CSCI699': 'CSCI699 Robotic Perception (Robotics)'}

In [ ]:
def two_queries_prompt(prev_query, curr_query, courses):
  courses = "\n".join(courses.values())
  question_prompt = f"""
Given the previous and current user queries, determine whether the current query pertains to the same course as the previous query.

Previous Query: {prev_query}
Current Query: {curr_query}

Courses:
{courses}

Rules:
If the user did not specify a course in the current query, then the user is asking about the same course as the previous query.
If the user specified a course in the current query, then the user is asking about a different course.
Give your answer as a boolean value.
"""
  return question_prompt

In [ ]:
def two_queries_prompt_cot(prev_query, curr_query, courses):
  courses = "\n".join(courses.values())
  question_prompt = f"""
Given the previous and current user queries, determine whether the current query pertains to the same course as the previous query.

Previous Query: {prev_query}
Current Query: {curr_query}

Courses:
{courses}

Rules:
If the user did not specify a course in the current query, then the user is asking about the same course as the previous query.
If the user specified a course in the current query, then the user is asking about a different course.

Examples:
1. Previous Query: Who is the instructor of CSCI544?
  Current Query: When is the final exam?
  Answer: True

2. Previous Query: Who is the instructor of CSCI544?
  Current Query: Who is the instructor of CSCI566?
  Answer: False

3. Previous Query: Who is the instructor?
  Current Query: How much does the midterm makes up the grade?
  Answer: True

Provide your answer as a boolean value.
"""
  return question_prompt

In [ ]:
def two_queries_prompt_one_shot(prev_query, curr_query, courses):
  courses = "\n".join(courses.values())
  question_prompt = f"""
Given the previous and current user queries, determine whether the current query pertains to the same course as the previous query.

Previous Query: {prev_query}
Current Query: {curr_query}

Courses:
{courses}

Rules:
If the user did not specify a course in the current query, then the user is asking about the same course as the previous query.
If the user specified a course in the current query, then the user is asking about a different course.

Example:
  Previous Query: Who is the instructor?
  Current Query: How much does the midterm makes up the grade?
  Answer: True

Provide your answer as a boolean value.
"""
  return question_prompt

In [ ]:
def query_history_prompt_cot(prev_queries, curr_query, courses):
  courses = "\n".join(courses.values())
  queries = "\n".join(prev_queries)
  question_prompt = f"""
Given the previous query history and the current user query, determine whether the current query pertains to the same course as the **last** previous query.

Previous Queries:
{queries}

Current Query: {curr_query}

Courses:
{courses}

Rules:
If the user did not specify a course in the current query, then the user is asking about the same course as the previous query.
If the user specified a course in the current qery, then the user is asking about a different course.

Examples:
1. Previous Query: Who is the instructor of CSCI544?
  Current Query: When is the final exam?
  Answer: True

2. Previous Query: Who is the instructor of CSCI544?
  Current Query: Who is the instructor of CSCI566?
  Answer: False

3. Previous Query: Who is the instructor?
  Current Query: How much does the midterm makes up the grade?
  Answer: True

Provide your answer as a boolean value.
"""
  return question_prompt

In [ ]:
def binary_prompt_cot(query, courses):
  courses = "\n".join(courses.values())
  question_prompt = f"""
Given the user query, determine whether the query specifies a course or not.

Query: {query}

Courses:
{courses}

Rules:
1. A query specifies a course if it explicitly mentions the course name, code, or a clear reference (e.g., "the database course").
2. If no course is mentioned, the query does not specify a course.

Examples:
1. Query: Who is the instructor of CSCI544?
  Answer: True

2. Query: When is the final exam?
  Answer: False

3. Query: Where is the classroom of the database course?
  Answer: True

Provide your answer as a boolean value.
"""
  return question_prompt


In [ ]:
prev_queries = ["Who is the instructor of CSCI544?", "When is the final exam?", "Who is the instructor of CSCI566?"]
curr_query = "How much does the midterm makes up the grade?"

prev = "When is the first homework due?"
cur = "When is the final exam?"

question_prompt = two_queries_prompt(prev, cur, full_course_info)
question_prompt_cot = two_queries_prompt_cot(prev, cur, full_course_info)
question_prompt_history = query_history_prompt_cot(prev_queries, curr_query, full_course_info)
question_prompt_binary = binary_prompt_cot(curr_query, full_course_info)
question_prompt_one_shot = two_queries_prompt_one_shot(prev, cur, full_course_info)


In [ ]:
print(question_prompt_one_shot)


Given the previous and current user queries, determine whether the current query pertains to the same course as the previous query.

Previous Query: When is the first homework due?
Current Query: When is the final exam?

Courses: 
CSCI544 Applied Natural Language Processing (NLP)
CSCI566 Deep Learning and its Applications (DL)
CSCI585 Database Systems (database)
CSCI596 Scientific Computing and Visualization
CSCI599 Distributed Systems
CSCI626 Text as Data
CSCI677 Advanced Computer Vision (CV)
CSCI699 Robotic Perception (Robotics)

Rules: 
If the user did not specify a course in the current query, then the user is asking about the same course as the previous query.
If the user specified a course in the current query, then the user is asking about a different course.

Example:
  Previous Query: Who is the instructor?
  Current Query: How much does the midterm makes up the grade?
  Answer: True

Provide your answer as a boolean value.



In [ ]:
def get_response(prompt):

  message = [
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
  ]

  response = together.chat.completions.create(
      model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
      messages=message,
      max_tokens=1024,
      temperature=0,
  )
  return response.choices[0].message.content

In [ ]:
def extract_bool(response: str):
  match = re.search(r'\b(True|False)\b', response, re.IGNORECASE)
  if match:
    return match.group(1).lower() == 'true'
  return "nan"

In [ ]:
response = get_response(question_prompt_one_shot)
print(response)

To determine whether the current query pertains to the same course as the previous query, I need to analyze the previous and current queries.

Previous Query: When is the first homework due?
Current Query: When is the final exam?

The previous query does not specify a course, but the current query does not specify a course either. However, the current query does not contain any keywords that match the course names. 

Since the current query does not specify a course and does not contain any keywords that match the course names, I will assume that the user is asking about the same course as the previous query.

Answer: True


In [ ]:
print(extract_bool(response))

True


In [ ]:
test_path = "/content/drive/MyDrive/CSCI544/Project/dataset/classifier/test.csv"
test_path_binary = "/content/drive/MyDrive/CSCI544/Project/dataset/classifier/combined_shuffled_binary.csv"
write_path_cot = "/content/drive/MyDrive/CSCI544/Project/results/test_results_cot.csv"
write_path_reg = "/content/drive/MyDrive/CSCI544/Project/results/test_results_reg.csv"
write_path_binary = "/content/drive/MyDrive/CSCI544/Project/results/test_results_binary.csv"
write_path_one_shot = "/content/drive/MyDrive/CSCI544/Project/results/test_results_one_shot.csv"

with open(test_path, "r") as f:
  df = pd.read_csv(f)

with open(test_path_binary, "r") as f:
  df_binary = pd.read_csv(f)

prev_queries = df["query1"].tolist()
curr_queries = df["query2"].tolist()
resDF = cp.deepcopy(df)
resDF["prediction"] = np.nan

queries = df_binary["query"].tolist()
resDF_binary = cp.deepcopy(df_binary)
resDF_binary["prediction"] = np.nan

In [ ]:
for prev, curr in tqdm(zip(prev_queries, curr_queries), desc="Processing Queries", total=len(prev_queries)):
  question_prompt = two_queries_prompt_one_shot(prev, curr, full_course_info)
  response = get_response(question_prompt)
  pred = extract_bool(response)
  resDF.loc[(resDF["query1"] == prev) & (resDF["query2"] == curr), "prediction"] = pred

Processing Queries:   0%|          | 0/1000 [00:00<?, ?it/s]<ipython-input-18-05cb0c7f32fe>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  resDF.loc[(resDF["query1"] == prev) & (resDF["query2"] == curr), "prediction"] = pred
Processing Queries: 100%|██████████| 1000/1000 [22:01<00:00,  1.32s/it]


In [ ]:
# test case for Single Query Binary w. 3 shot

# for q in tqdm(queries, desc="Processing Queries", total=len(queries)):
#   question_prompt = binary_prompt_cot(q, full_course_info)
#   response = get_response(question_prompt)
#   pred = extract_bool(response)
#   resDF_binary.loc[resDF_binary["query"] == q, "prediction"] = pred

In [ ]:
with open(write_path_one_shot, "wb") as f:
  resDF.to_csv(f, index=False)

In [ ]:
def accuracy(path):
  with open(path, "r") as f:
    df = pd.read_csv(f)
  rate = (df["prediction"] == df["label"]).mean()
  return rate

In [ ]:
binary = accuracy(write_path_binary)
three_shot = accuracy(write_path_cot)
zero_shot = accuracy(write_path_reg)
one_shot = accuracy(write_path_one_shot)
print(f"Binary Accuracy: {binary}")
print(f"COT Accuracy: {three_shot}")
print(f"Regular Accuracy: {zero_shot}")
print(f"One Shot Accuracy: {one_shot}")

Binary Accuracy: 0.8770491803278688
COT Accuracy: 0.898
Regular Accuracy: 0.834
One Shot Accuracy: 0.868


In [ ]:
x = pd.DataFrame({"Model": ["Single Query Binary 3 shot", "Prev Curr Query 3 shot", "Prev Curr Query 1 shot" ,"Prev Curr Query 0 shot"], "Accuracy": [binary, three_shot, one_shot, zero_shot]})

In [ ]:
x

,Model,Accuracy
0,Single Query Binary 3 shot,0.877049
1,Prev Curr Query 3 shot,0.898000
2,Prev Curr Query 1 shot,0.868000
3,Prev Curr Query 0 shot,0.834000
